Implemented on top of AzureML.
Reference: https://docs.microsoft.com/en-us/azure/machine-learning/

Pipelines are going to be created inside some Azure ML workspace.
If you already have one, you can get the reference:

In [1]:
# if you already have a workspace created, just run this to initialize workspace config
# if you haven't created a workspace, skip this
from azureml.core import Workspace

ws = Workspace.from_config()

Found the config file in: C:\dev\reinforcement_learning\batch\aml_config\config.json


In case if you do not have, any you can either create it on Azure portal or programmatically:

In [ ]:
# if you don't have a workspace already created then run this step
from azureml.core import Workspace
ws = Workspace.create(name = "cheng_dashboard_tutorial_test",
                      subscription_id = "9a2d7383-3c7d-492c-94fc-ba65be408672",
                      resource_group = "cps-dev-cheng",
                      create_resource_group = False,
                      location = 'eastus'
                     )
ws.write_config()
ws.get_details()

This is a library of pipeline steps / primitives for creating batch training pipelines from logs produced by personalization service. Also, you can find some recipe pipelines which can be either used as-is or as a reference.

For creating pipeline application context is required:

In [ ]:
ws.datastores # list out all datastores

In [2]:
import application
from application import context

# set up where to read the log files
ctx = context.Context(
    accountName = 'dstutorial01eus',
    accountKey = 'TaaDrF3p5gU6uwDlz557B95SqciyFOAI5cBke12VsuIN5ptbxfzMrWcHPKiQnc85ptlFa5nYRP1TtgLbdCsykw==',
    appId = 'tutorial-002',
    appFolder = '20181012202114'
)

Now you can create batch training pipeline for vw batch training:

Created pipeline can be either submitted for execution:

Or published to be available via REST endpoint:

Example of REST call:
Reference for better understanding of Azure ML authentication:
https://github.com/Azure/MachineLearningNotebooks/blob/f16bf27e26d6ff7a2cf76e8cbcda4ccc9c878a0c/how-to-use-azureml/manage-azureml-service/authentication-in-azureml/authentication-in-azure-ml.ipynb

In [3]:
from pipelines import extract_pipeline
extractPipeline = extract_pipeline.create_pipeline(ws, ctx, parallel_jobs = 2) # I only parallel running 2 jobs for now for debugging purpose

Found existing cluster: python
Best command selection step is created successfully
Data store for the app context {dstutorial01eustutorial002} is found
Found existing cluster: extractor
Cache step is successfully created
Found existing cluster: vw
Found existing cluster: python
Vw sweep step is successfully created
['--cache_folder', $AZUREML_DATAREFERENCE_Cache, '--model_folder', $AZUREML_DATAREFERENCE_models, '--output_folder', $AZUREML_DATAREFERENCE_Predictions, '--command_folders', $AZUREML_DATAREFERENCE_Hyper1_best_commands]
==inputs==
[$AZUREML_DATAREFERENCE_Cache, $AZUREML_DATAREFERENCE_Hyper1_best_commands]
Found existing cluster: vw-predict
Vw predict step is successfully created
Data store for the app context {dstutorial01eustutorial002} is found
Found existing cluster: dashboard
Dashboard step is successfully created
extractPipeline is succesfully created.
Step Predict is ready to be created [9adc7685]
extractPipeline is succesfully validated.


In [4]:
# pipeline params set up the date range of logs

from azureml.core import Experiment  
regen_outputs = False
extractExperiment = Experiment(ws, 'tutorial_with_quad').submit(
    extractPipeline, 
    pipeline_params = {
        "start_date":'10/12/2018',
        "end_date":'10/12/2018'
    }, 
    regenerate_outputs = regen_outputs
)
print("Extract Pipeline is submitted for execution")
from azureml.widgets import RunDetails
RunDetails(extractExperiment).show()

Created step Prepare command [96ce223c][8dcbde83-cf1d-4ee3-b2a3-c3e7688f71f9], (This step is eligible to reuse a previous run's output)
Created step Cache [ae9dc5be][613777aa-4c13-4cbb-a214-287630251da2], (This step is eligible to reuse a previous run's output)
Created step Predict [9adc7685][79c6b087-d7c8-40ad-9c66-99372478520c], (This step will run and generate new outputs)
Created step Best command [aece9349][2164f51d-1fdb-4568-a67c-5e2e0f4ec1bd], (This step is eligible to reuse a previous run's output)
Created step Sweep [vw] [8c0fad3e][6c5fea11-55f6-4875-bcb9-fda04dc13c2e], (This step will run and generate new outputs)
Created step Dashboard [0582d2d5][dcebc69c-c859-41d4-9cab-7a645643e3d5], (This step is eligible to reuse a previous run's output)
Using data reference Application_logs for StepId [552d346f][e4e1ba10-3080-46c7-9e73-a15913e89bcc], (Consumers of this data are eligible to reuse prior runs.)
Using data reference Application_logs for StepId [d7a5255a][e4e1ba10-3080-46c7-9

_PipelineWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', …